# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.
%load_ext dotenv
%dotenv 



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Write your code below.
price_data = glob(os.path.join(os.getenv('SRC_DIR'), "data","prices","*","*","*.parquet"))

len(price_data)


2859

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.

dd_px = dd.read_parquet(price_data, engine="pyarrow", gather_statistics=False)

#print(dd_px.head())

# Apply groupby for lag features
dd_feat = dd_px.groupby("ticker", group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1     = x["Close"].shift(1),
        Adj_Close_lag_1 = x["Adj Close"].shift(1),
        returns         = (x["Close"] / x["Close"].shift(1)) - 1,
        hi_lo_range     = x["High"] - x["Low"],
    )
)

c:\Users\semih\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\io\parquet\core.py:428: FutureWarning: ``gather_statistics`` is deprecated and will be removed in a future release. Please use ``calculate_divisions`` instead.
  warnings.warn(


             Date   Open   High    Low  Close  Adj Close      Volume   source  \
137315 2001-07-19  15.10  15.29  15.00  15.17  11.404394  34994300.0  ACN.csv   
137316 2001-07-20  15.05  15.05  14.80  15.01  11.284108   9238500.0  ACN.csv   
137317 2001-07-23  15.00  15.01  14.55  15.00  11.276587   7501000.0  ACN.csv   
137318 2001-07-24  14.95  14.97  14.70  14.86  11.171341   3537300.0  ACN.csv   
137319 2001-07-25  14.70  14.95  14.65  14.95  11.238999   4208100.0  ACN.csv   

       ticker  Year  
137315    ACN  2001  
137316    ACN  2001  
137317    ACN  2001  
137318    ACN  2001  
137319    ACN  2001  


c:\Users\semih\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\groupby.py:2454: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  meta = self._meta_nonempty.apply(func, *meta_args, **meta_kwargs)
C:\Users\semih\AppData\Local\Temp\ipykernel_19444\586209268.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby("ticker", group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

pd_table = dd_feat.reset_index().compute()
print(type(pd_table))
print(pd_table.columns.tolist())
print(pd_table.head())
assert "returns" in pd_table.columns


<class 'pandas.core.frame.DataFrame'>
['index', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'source', 'ticker', 'Year', 'Close_lag_1', 'Adj_Close_lag_1', 'returns', 'hi_lo_range']
   index       Date       Open       High        Low      Close  Adj Close  \
0     28 2006-11-21  26.799999  26.840000  26.799999  26.840000  24.354116   
1     29 2006-11-22  26.950001  26.950001  26.870001  26.889999  24.399477   
2     30 2006-11-24  26.889999  26.889999  26.889999  26.889999  24.399477   
3     31 2006-11-27  26.370001  26.370001  26.309999  26.309999  23.873203   
4     32 2006-11-28  26.309999  26.309999  26.309999  26.309999  23.873203   

   Volume   source ticker  Year  Close_lag_1  Adj_Close_lag_1   returns  \
0   500.0  PRN.csv    PRN  2006          NaN              NaN       NaN   
1  1500.0  PRN.csv    PRN  2006    26.840000        24.354116  0.001863   
2     0.0  PRN.csv    PRN  2006    26.889999        24.399477  0.000000   
3  2300.0  PRN.csv    PRN  2006 

In [18]:
ppd_table = pd_table.sort_values(["ticker", "Date"])
# Add 10-day moving average of returns per ticker
pd_table["returns_ma10"] = (
    pd_table.groupby("ticker")["returns"]
          .transform(lambda s: s.rolling(10, min_periods=1).mean())
)

# Quick check
print(pd_table[["ticker", "Date", "returns", "returns_ma10"]].head(15))

   ticker       Date   returns  returns_ma10
0     PRN 2006-11-21       NaN           NaN
1     PRN 2006-11-22  0.001863      0.001863
2     PRN 2006-11-24  0.000000      0.000931
3     PRN 2006-11-27 -0.021569     -0.006569
4     PRN 2006-11-28  0.000000     -0.004927
5     PRN 2006-11-29  0.012163     -0.001509
6     PRN 2006-11-30  0.000000     -0.001257
7     PRN 2006-12-01 -0.010514     -0.002580
8     PRN 2006-12-04  0.018975      0.000115
9     PRN 2006-12-05  0.011173      0.001343
10    PRN 2006-12-06  0.000000      0.001209
11    PRN 2006-12-07 -0.007735      0.000249
12    PRN 2006-12-08  0.004454      0.000695
13    PRN 2006-12-11 -0.004435      0.002408
14    PRN 2006-12-12  0.000000      0.002408


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

I think it is necessary. I realized there are 2 parquet files in our price folder. This is because dask partitions our dataset. The moving window function works better in a continuous dataset.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.